<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Send template maintainer monthly report
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Send+template+maintainer+monthly+report:+Error+short+description">Bug report</a>

**Tags:** #github #issues #merged #rest #api #snippet #operations #email #awesomenotebooks #maintainer

**Author:** [Benjamin Filly](https://www.linkedin.com/in/benjamin-filly-05427727a/)

**Last update:** 2023-07-24 (Created: 2023-07-24)

**Description:** This notebook retrieves data to ascertain the sponsorships provided by Naas for template maintainers and dispatches a notification on every 7th day of the month, as well as the last three days. It incorporates the monthly count of issue closed with estimates, and the number of Pull Requests reviewed within the month.

**References:**
- [GitHub REST API Documentation](https://docs.github.com/en/rest/issues/issues?apiVersion=2022-11-28#update-an-issue)
- [os](https://docs.python.org/3/library/os.html)
- [json](https://docs.python.org/3/library/json.html)
- [datetime](https://docs.python.org/3/library/datetime.html)
- [pandas](https://pandas.pydata.org/docs/)
- [requests](https://docs.python-requests.org/en/latest/)

## Input

### Import libraries

In [ ]:
import pandas as pd
from github import Github
from datetime import datetime, timezone
import requests
from bs4 import BeautifulSoup
from IPython.display import display, HTML
import numpy as np
from datetime import datetime
import naas
from naas_drivers import naasauth, emailbuilder
import warnings
warnings.filterwarnings("ignore")

### Setup variables
**Mandatory**

- `github_token`: [GitHub token](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line)
- `contributor_profile`: GitHub username of the contributor

**Optional**

- `repo_name`: name of the repository in two part: owner_name/repository_name
- `estimates_view_url`:Url of the project where the estimated are located
- `cron`: cron params for naas scheduler change it using [Crontab](https://crontab.guru/)
- `email_to`: This variable is used for storing a list of email addresses that will receive the notification email
- `email_from`:  Email sender: Replace with your email account or notification@naas.ai
- `subject`: Email subject

In [ ]:
# Mandatory
github_token = naas.secret.get("GITHUB_TOKEN") or "YOUR_GITHUB_TOKEN"
contributor_profile = "FlorentLvr"

# Optional
scenario = "This Month" #"Last month"
repo_name = "jupyter-naas/awesome-notebooks" #Example: jupyter-naas/awesome-notebooks
estimates_view_url = "https://github.com/orgs/jupyter-naas/projects/10/views/20" #example: https://github.com/orgs/jupyter-naas/projects/10/views/20
cron = "0 0 7,14,21,28,30,31 * *" # At 00:00 on every day-of-month from 28 through 31
email_to = [naasauth.connect().user.me().get("username")] # List to emails address of the receiver(s)
email_from = "notification@naas.ai"
subject = "Templates Maintainer Monthly Report"

## Model

### Connect to GitHub repo

In [ ]:
# Connect to the GitHub API using pygithub library
g = Github(github_token)

# Get the repository
repo = g.get_repo(repo_name)

### Get PRs merged on scenario

In [ ]:
# Create scenario to filter PRs
def get_scenario(scenario):
    # Init
    current_date = datetime.now(timezone.utc)
    date_scenario = current_date
    
    # Get date from scenario
    if scenario == "Last Month":
        month_scenario = current_date.month - 1  
        date_scenario = current_date.replace(month=month_scenario)
    return date_scenario

scenario_filter = get_scenario(scenario)

# Get PRs closed
pull_requests = repo.get_pulls(state='closed', sort='updated', direction='desc')
print("✅ Pull Requests closed:", pull_requests.totalCount)

# Get PRs merged on scenario
merged_pr = []
for pr in pull_requests:
    # Get PRs merged
    if pr.merged_at:
        merged_at = pr.merged_at.strftime("%Y%m")
        
        # Get PRs merged on scenario
        if int(merged_at) == int(scenario_filter.strftime("%Y%m")):
            merged_pr.append(pr)
        if int(merged_at) < int(scenario_filter.strftime("%Y%m")):
            break
            
print(f"📌 Pull Requests merged on {scenario_filter.strftime('%Y-%m')}:", len(merged_pr))

### Get PRs assigned and reviewed by contributor

In [ ]:
assigned_pr = []
reviewed_pr = []

for pr in merged_pr:
    # Get PRs assigned
    if pr.assignee and pr.assignee.login == contributor_profile:
        assigned_pr.append(pr)
        
    # Get PRs reviewed
    if pr.requested_reviewers:
        for r in pr.requested_reviewers:
            if r.login == contributor_profile:
                reviewed_pr.append(pr)

print(f"🧑‍💻 Pull Requests assigned:", len(assigned_pr))
print(f"👀 Pull Requests reviewed:", len(reviewed_pr))

### Get estimates from project view

In [ ]:
# Init
data_bs4 = []

# Get HTML from URL
response = requests.get(estimates_view_url)
html = response.text

# Parse HTML
soup = BeautifulSoup(html, "html.parser")

# Get cards
elements = soup.find_all("script", {"id": "memex-items-data"})

# Iterate over the elements and split their text
for element in elements:
    text = element.text
    split_text = text.split('{"contentId":')[1:]  # Split the text as needed
    
    # Split the soup for each element
    for s in split_text:
        s = s.split('"memexProjectColumnId":')[1:]
        # Get the values using splits
        title = s[0].split('"raw":"')[-1].split('"')[0]
        issue_number = s[0].split('"number":')[-1].split(',')[0]
        assignees = s[1].split('"login":"')[-1].split('"')[0]
        PR_url = s[2].split('"url":"')[-1].split('"')[0]
        estimate = s[3].split('"value":')[-1].split('}')[0]
        
        # Handle possible error
        if not str(issue_number).isdigit():
            issue_number = "❌ Error"
        
        # Create a dictionary with the values
        tmp_bs4 = {
            "Title": title,
            "Issue Number": issue_number,
            "Assignees": assignees,
            "PR URL": PR_url,
            "Estimate": estimate,
        }
        # Append the dictionary to the data list
        data_bs4.append(tmp_bs4)

# Create a DataFrame from the data list
df_estimates = pd.DataFrame(data_bs4)     
print("✅ Row fetched on estimates view:", len(df_estimates))
# df_estimates.head(1)

### Calculate estimates on issue closed

In [ ]:
# Store the relevant information in a DataFrame
data = []
total_estimate = 0
filtered_df_init = pd.DataFrame()

for pr in assigned_pr:
    tmp = {
        "title": pr.title,
        "number": pr.number,
        "url": pr.html_url,
        "assignee": pr.assignee.login,
        "created_at": pr.created_at.isoformat(),
        "updated_at": pr.updated_at.isoformat(),
        "merged_at": pr.merged_at.isoformat(),
    }
    data.append(tmp)
df_pr = pd.DataFrame(data)

if len(df_pr) > 0:
    # Filter df_init with the URLs from df_pr
    filtered_df_init = df_estimates[df_estimates["PR URL"].isin(df_pr["url"])]
    filtered_df_init['Estimate'] = filtered_df_init['Estimate'].str.replace("null", "0")

    # Set the display option for max column width to ensure the link is fully displayed
    pd.set_option('display.max_colwidth', None)

    # Convert the "PR URL" column values into clickable links using HTML formatting
    filtered_df_init["PR URL"] = filtered_df_init["PR URL"].apply(lambda x: f'<a href="{x}">{x}</a>')

    # Calculate the total of the "Estimate" column and convert it to an integer
    total_estimate = int(filtered_df_init["Estimate"].astype(float).sum())

    # Display the DataFrame with clickable links and format the "Estimate" column as integers
    filtered_df_init["Estimate"] = filtered_df_init["Estimate"].astype(float).astype(int)
    filtered_df_init["Email_List"] = filtered_df_init["Title"] + ": " + filtered_df_init["Estimate"].astype(str)
    display(HTML(filtered_df_init.to_html(escape=False, index=False)))

# Separate sentence with emoji for the total estimate
print(f"\n🚀 The total estimate is: {total_estimate}")

### Creating the E-mail
We used `Naas_emailbuilder_demo.ipynb` to create our email.

In [ ]:
# Sample logo URL
logo_img = "https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=186"

# Get the current date
scenario_display = scenario_filter.strftime("%B %Y")

# Convert the "Estimate" column values to string
sum_estimates = 0
list_issues = []
if len(filtered_df_init) > 0:
    sum_estimates = filtered_df_init["Estimate"].astype(int).sum()
    list_issues = [emailbuilder.link(row["PR URL"], row["Email_List"]) for index, row in filtered_df_init.iterrows()]
list_prs = [emailbuilder.link(pr.html_url, pr.title) for pr in reviewed_pr]

reward_issues_merged = sum_estimates * 5
reward_prs_reviewed = len(list_prs) * 10
rewards =  reward_issues_merged + reward_prs_reviewed

# Define the content for the email using the email builder
email_content = {
#     "element": naas_drivers.emailbuilder.title(f'<div {center_style}>📝 Report on templates created for {contributor_profile}</div>'),
    "heading": emailbuilder.heading(f'Monthly Report - {scenario_display}'),
    "intro": emailbuilder.text(f"Hi {contributor_profile},"),
    "reward": emailbuilder.text(f'💵 Here is the reward your contribution as templates maintainer: {rewards}$'),
    "reward_det": emailbuilder.list([f"Issues closed by PRs merged: {reward_issues_merged} $", f"PRs reviewed: {reward_prs_reviewed} $"]),
    "contrib": emailbuilder.text(f'Please find below the detail of your contributions.'),
    "assign": emailbuilder.text(f'🧑‍💻 Issues closed by PRs merged: {len(filtered_df_init)} (Total estimates: {sum_estimates})'),
    "assign_list": emailbuilder.list(list_issues),
    "review": emailbuilder.text(f'👀💻 PRs reviewed: {len(reviewed_pr)}'),
    "review_list": emailbuilder.list(list_prs),
    "footer": emailbuilder.footer_company(naas=True),
}

# Generate the email content as HTML
content = emailbuilder.generate(display="iframe", **email_content)

## Output

### Send email

In [ ]:
# Send the email using naas.notification
naas.notification.send(email_to=email_to, subject=subject, html=content, email_from=email_from)

### Add scheduler

In [ ]:
naas.scheduler.add(cron=cron)